# Klarna Case Study

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge


## Data Retrieval

In [2]:
def get_col_info():
    colInfo = 'CaseStudyCols.csv'

    with open(colInfo, newline='') as csv_file:
        reader = csv.reader(csv_file, delimiter=';')
        # skip over the first row
        reader.__next__()
        rows = list(reader)

    col_names_types = {row[0]: row[1] for row in rows}

    return col_names_types


In [3]:
dataSet = 'dataset.csv'

"""
Note that in reality the column 'default' and
'worst_status_active_inv' as actually ints, but
since they have NA values, we load them as objects
for now.

But note that the entries where default = NA are actually our
entries that we are supposed to predict (validate set?)
"""

df = pd.read_csv(dataSet, delimiter=';', dtype = get_col_info(), keep_default_na=True)

Data Processing & Wrangling

In [4]:
# Strip out the NA entries for the column 'default'
# First we save the entries to a seperate dataframe

defaults_df = df[df['default'].isna().copy()]

df = df[df['default'].notna().copy()]
df['default'] = pd.to_numeric(df['default']).astype('int32')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89976 entries, 0 to 89975
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   uuid                                 89976 non-null  object 
 1   default                              89976 non-null  int32  
 2   account_amount_added_12_24m          89976 non-null  int32  
 3   account_days_in_dc_12_24m            79293 non-null  float64
 4   account_days_in_rem_12_24m           79293 non-null  float64
 5   account_days_in_term_12_24m          79293 non-null  float64
 6   account_incoming_debt_vs_paid_0_24m  36619 non-null  float64
 7   account_status                       41042 non-null  float64
 8   account_worst_status_0_3m            41042 non-null  float64
 9   account_worst_status_12_24m          29921 non-null  float64
 10  account_worst_status_3_6m            38038 non-null  float64
 11  account_worst_status_6_12m  

## EDA

## Columns with higher percentage of NA

In [5]:
# Explore the dataset, in particular look at the NA counts
has_na = []
for col in df.columns:
    perc_na = 0
    perc_na = round(df[col].isna().sum()/len(df), 2)
    if perc_na > 0:
        has_na.append([col, perc_na])


# Plot out the columns with NA (their percentages)
# fig, ax = plt.subplots(figsize=(10, 5), dpi=100)
# na_plot = sns.barplot(x=[n[0] for n in has_na], y=[n[1] for n in has_na],  ax=ax)
# na_plot.set_xticklabels(na_plot.get_xticklabels(), rotation=90, horizontalalignment='right')
# na_plot.set_title("% of values = NA");


In [6]:
## Let's explore some of the columns (other than default)
## Look in particular at the columns with high NA.
## Do we need those columns or can we simply drop them?

In [7]:
## Make a list of columns that have high % NA
# % NA threshhold for columns to ignore
na_threshold = .5
cols_to_exclude = [col[0] for col in has_na if col[1] > na_threshold]

In [8]:
## We can see that 'merchant_category', 'merchant_group'
## and 'name_in_email' are categorical (object = string)
## These will either need to be ignored (not a good idea)
## or encoded.

In [9]:
# Have a look at the two 'merchant_' columns
for c in df.columns:
    if(c[:8] == 'merchant'):
            print(df[c].value_counts())
df.info()

Diversified entertainment                                  34781
Youthful Shoes & Clothing                                  10524
Books & Magazines                                           8447
General Shoes & Clothing                                    4162
Concept stores & Miscellaneous                              3969
Sports gear & Outdoor                                       3356
Dietary supplements                                         2777
Diversified children products                               2691
Diversified electronics                                     1627
Prints & Photos                                             1502
Children Clothes & Nurturing products                       1357
Pet supplies                                                1185
Electronic equipment & Related accessories                   946
Hobby articles                                               817
Jewelry & Watches                                            815
Prescription optics      

## Look for correlations

In [10]:
def corr_matrix(df):
    correlation_matrix = df.corr()
    column_names = correlation_matrix.columns

    # Convert the correlation matrix into a DataFrame
    corr_df = correlation_matrix.stack().reset_index()

    # Rename the columns
    corr_df.columns = ['feature_1','feature_2', 'correlation']

    # Remove "self correlations"
    no_self_correlation = (corr_df['feature_1'] != corr_df['feature_2'])
    corr_df = corr_df[no_self_correlation]

    # Absolute correlation
    corr_df['absolute_correlation'] = np.abs(corr_df['correlation'])

    # Correlation by pairs of features
    return corr_df.sort_values(by="absolute_correlation", ascending=False)

corr_matrix(df)

,feature_1,feature_2,correlation,absolute_correlation
621,max_paid_inv_0_24m,max_paid_inv_0_12m,0.889912,0.889912
584,max_paid_inv_0_12m,max_paid_inv_0_24m,0.889912,0.889912
818,num_arch_ok_0_12m,num_arch_ok_12_24m,0.864148,0.864148
855,num_arch_ok_12_24m,num_arch_ok_0_12m,0.864148,0.864148
1245,status_max_archived_0_24_months,status_max_archived_0_12_months,0.842982,0.842982
...,...,...,...,...
75,account_amount_added_12_24m,time_hours,-0.000061,0.000061
1257,recovery_debt,account_incoming_debt_vs_paid_0_24m,-0.000055,0.000055
222,account_incoming_debt_vs_paid_0_24m,recovery_debt,-0.000055,0.000055
1413,time_hours,account_worst_status_3_6m,0.000025,0.000025


In [11]:
## There are already 36 columns in the original dataset. A bit less after we
## removed the high percentage NA columns. Before getting into the pipeline
## let's remove some columns that are probably not needed.

## First, columns that are highly correlated (corr > .8)
cols_to_exclude +=  ['max_paid_inv_0_24m', 'num_arch_ok_12_24m', 'status_max_archived_0_24_months']

## Next, categorical columns that 'probably' don't add much value
## We still have the 'merchant_group' column that should be usefule
cols_to_exclude += ['merchant_category', 'name_in_email']

## Pipeline

In [12]:
cols_to_exclude += ['default', 'uuid']

# Defining the features and the target
X = df.drop(columns=cols_to_exclude)
y = df['default']

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((71980, 29), (17996, 29), (71980,), (17996,))

In [23]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71980 entries, 70617 to 17399
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   account_amount_added_12_24m       71980 non-null  int32  
 1   account_days_in_dc_12_24m         63420 non-null  float64
 2   account_days_in_rem_12_24m        63420 non-null  float64
 3   account_days_in_term_12_24m       63420 non-null  float64
 4   age                               71980 non-null  int32  
 5   avg_payment_span_0_12m            54865 non-null  float64
 6   avg_payment_span_0_3m             36553 non-null  float64
 7   merchant_group                    71980 non-null  object 
 8   has_paid                          71980 non-null  bool   
 9   max_paid_inv_0_12m                71980 non-null  float64
 10  num_active_div_by_paid_inv_0_12m  55525 non-null  float64
 11  num_active_inv                    71980 non-null  int32  
 12  

In [13]:
# Build the pipeline with the different steps
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('standard_scaler', StandardScaler())
])

In [14]:
pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('standard_scaler', StandardScaler())])

In [15]:
# pipeline.fit(X_train[['age']])
# pipeline.transform(X_train[['age']])

In [16]:
# Impute then scale numerical values:
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),
    ('standard_scaler', StandardScaler())
])

# Encode categorical values
cat_transformer = OneHotEncoder(handle_unknown='ignore')

# Parallelize "num_transformer" and "cat_transfomer"
numeric_cols = X.select_dtypes(include='number').describe().columns.to_list()
cat_cols = X.select_dtypes(include='object').describe().columns.to_list()

preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, numeric_cols),
    ('cat_transformer', cat_transformer, cat_cols),
] )

In [17]:
preprocessor

ColumnTransformer(transformers=[('num_transformer',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('standard_scaler',
                                                  StandardScaler())]),
                                 ['account_amount_added_12_24m',
                                  'account_days_in_dc_12_24m',
                                  'account_days_in_rem_12_24m',
                                  'account_days_in_term_12_24m', 'age',
                                  'avg_payment_span_0_12m',
                                  'avg_payment_span_0_3m', 'max_paid_inv_0_12m',
                                  'num_active_div_by_pai...
                                  'status_last_archived_0_24m',
                                  'status_2nd_last_archived_0_24m',
                                  'status_3rd_last_archived_0_24m',
                                  'status_max_archived_0_6_months',
                                  'status_max_archived_0_12_months',
                                  'recovery_debt',
                                  'sum_capital_paid_account_0_12m',
                                  'sum_capital_paid_account_12_24m',
                                  'sum_paid_inv_0_12m', 'time_hours']),
                                ('cat_transformer',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['merchant_group'])])

In [18]:
X_train_transformed = preprocessor.fit_transform(X_train)

print("Original training set")
display(X_train.head(3))

print("Preprocessed training set")
display(pd.DataFrame(X_train_transformed).head(3))

Original training set


,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,age,avg_payment_span_0_12m,avg_payment_span_0_3m,merchant_group,has_paid,max_paid_inv_0_12m,...,status_last_archived_0_24m,status_2nd_last_archived_0_24m,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours
70617,133492,0.0,69.0,0.0,23,NaN,NaN,"Leisure, Sport & Hobby",True,3000.0,...,1,1,1,0,0,0,106346,42575,6000,12.997500
73224,0,0.0,0.0,0.0,46,14.8,14.0,Entertainment,True,3480.0,...,1,1,1,1,1,0,0,0,18855,22.639167
37293,7335,0.0,0.0,0.0,22,NaN,NaN,"Leisure, Sport & Hobby",True,0.0,...,1,0,0,0,0,0,29622,7372,0,17.426944


Preprocessed training set


,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,3.409443,-0.041969,2.982171,-0.105361,-0.999531,-3.206945e-16,-2.437202e-16,-0.462570,1.080148e-16,-0.386196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.345427,-0.041969,-0.234912,-0.105361,0.773300,-2.843901e-01,-1.259421e-01,-0.426922,-4.456749e-01,-0.386196,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.139108,-0.041969,-0.234912,-0.105361,-1.076611,-3.206945e-16,-2.437202e-16,-0.685372,1.080148e-16,-0.386196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
transformed_df = pd.DataFrame(
    X_train_transformed,
    columns=preprocessor.get_feature_names_out()
)

transformed_df

,num_transformer__account_amount_added_12_24m,num_transformer__account_days_in_dc_12_24m,num_transformer__account_days_in_rem_12_24m,num_transformer__account_days_in_term_12_24m,num_transformer__age,num_transformer__avg_payment_span_0_12m,num_transformer__avg_payment_span_0_3m,num_transformer__max_paid_inv_0_12m,num_transformer__num_active_div_by_paid_inv_0_12m,num_transformer__num_active_inv,...,cat_transformer__merchant_group_Clothing & Shoes,cat_transformer__merchant_group_Electronics,cat_transformer__merchant_group_Entertainment,cat_transformer__merchant_group_Erotic Materials,cat_transformer__merchant_group_Food & Beverage,cat_transformer__merchant_group_Health & Beauty,cat_transformer__merchant_group_Home & Garden,cat_transformer__merchant_group_Intangible products,cat_transformer__merchant_group_Jewelry & Accessories,"cat_transformer__merchant_group_Leisure, Sport & Hobby"
0,3.409443,-0.041969,2.982171,-0.105361,-0.999531,-3.206945e-16,-2.437202e-16,-0.462570,1.080148e-16,-0.386196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.345427,-0.041969,-0.234912,-0.105361,0.773300,-2.843901e-01,-1.259421e-01,-0.426922,-4.456749e-01,-0.386196,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.139108,-0.041969,-0.234912,-0.105361,-1.076611,-3.206945e-16,-2.437202e-16,-0.685372,1.080148e-16,-0.386196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.575570,-0.041969,1.490190,-0.105361,-0.074576,-3.206945e-16,-2.437202e-16,-0.016967,-4.456749e-01,-0.386196,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.129601,-0.041969,-0.234912,-0.105361,-0.074576,-3.206945e-16,-2.437202e-16,-0.196322,-4.456749e-01,-0.386196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71975,-0.345427,-0.041969,-0.234912,-0.105361,1.929494,-5.371390e-01,4.914683e-01,-0.456629,-4.456749e-01,-0.386196,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71976,-0.345427,-0.041969,-0.234912,-0.105361,1.081618,-3.206945e-16,-2.437202e-16,-0.685372,1.080148e-16,-0.386196,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71977,-0.345427,-0.041969,-0.234912,-0.105361,-0.228735,1.990351e+00,-2.437202e-16,-0.085144,-4.456749e-01,-0.386196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71978,-0.345427,-0.041969,-0.234912,-0.105361,-0.691213,1.850009e-01,1.291815e+00,0.194101,-4.456749e-01,-0.386196,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [20]:
#The reality is that the scaling/encoding should not change the correlations
corr_matrix(transformed_df)

,feature_1,feature_2,correlation,absolute_correlation
801,num_transformer__status_max_archived_0_6_months,num_transformer__status_max_archived_0_12_months,0.734074,0.734074
839,num_transformer__status_max_archived_0_12_months,num_transformer__status_max_archived_0_6_months,0.734074,0.734074
24,num_transformer__account_amount_added_12_24m,num_transformer__sum_capital_paid_account_12_24m,0.710491,0.710491
936,num_transformer__sum_capital_paid_account_12_24m,num_transformer__account_amount_added_12_24m,0.710491,0.710491
721,num_transformer__status_2nd_last_archived_0_24m,num_transformer__status_3rd_last_archived_0_24m,0.691787,0.691787
...,...,...,...,...
377,num_transformer__num_active_inv,num_transformer__time_hours,-0.000051,0.000051
1258,cat_transformer__merchant_group_Erotic Materials,num_transformer__num_arch_dc_0_12m,0.000038,0.000038
422,num_transformer__num_arch_dc_0_12m,cat_transformer__merchant_group_Erotic Materials,0.000038,0.000038
71,num_transformer__account_days_in_dc_12_24m,cat_transformer__merchant_group_Erotic Materials,-0.000018,0.000018


In [21]:
# Add estimator
pipeline = make_pipeline(preprocessor, Ridge())
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('standard_scaler',
                                                                   StandardScaler())]),
                                                  ['account_amount_added_12_24m',
                                                   'account_days_in_dc_12_24m',
                                                   'account_days_in_rem_12_24m',
                                                   'account_days_in_term_12_24m',
                                                   'age',
                                                   'avg_payment_span_0_12m',
                                                   'avg_payment_span_0_3m',
                                                   'max_p...
                                                   'status_2nd_last_archived_0_24m',
                                                   'status_3rd_last_archived_0_24m',
                                                   'status_max_archived_0_6_months',
                                                   'status_max_archived_0_12_months',
                                                   'recovery_debt',
                                                   'sum_capital_paid_account_0_12m',
                                                   'sum_capital_paid_account_12_24m',
                                                   'sum_paid_inv_0_12m',
                                                   'time_hours']),
                                                 ('cat_transformer',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['merchant_group'])])),
                ('ridge', Ridge())])

In [22]:
# Train Pipeline
pipeline.fit(X_train,y_train)

# Make predictions
pipeline.predict(X_test.iloc[0:1])

# Score model
pipeline.score(X_test,y_test)

0.05221261370926977